## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數

Using TensorFlow backend.


In [2]:
num_classes = 10

# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

input_shape = x_train.shape[1:]

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# 對 label 做 one-hot encoding
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [3]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [4]:
n = 6
depth = 6 * n + 2

# 建立 ResNet 模型
model = resnet(input_shape, depth, num_classes) 
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()


Learning rate:  0.001
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
_____________________________________________________________________

In [5]:
lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [lr_reducer, lr_scheduler]
print('Using real-time data augmentation.')

Using real-time data augmentation.


In [6]:
augment_generator = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

In [7]:
batch_size = 64
epochs = 50
history = model.fit_generator(augment_generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=int(len(x_train)/batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    workers=4,
                    callbacks=callbacks)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/50
Learning rate:  0.001
781/781 [==============================] - 180s 230ms/step - loss: 0.4731 - accuracy: 0.9136 - val_loss: 0.3780 - val_accuracy: 0.9044
Epoch 2/50
Learning rate:  0.001
781/781 [==============================] - 182s 233ms/step - loss: 0.2742 - accuracy: 0.9294 - val_loss: 0.4645 - val_accuracy: 0.8864
Epoch 3/50
Learning rate:  0.001
781/781 [==============================] - 180s 230ms/step - loss: 0.2313 - accuracy: 0.9378 - val_loss: 0.3293 - val_accuracy: 0.9060
Epoch 4/50
Learning rate:  0.001
781/781 [==============================] - 181s 232ms/step - loss: 0.2075 - accuracy: 0.9432 - val_loss: 0.3748 - val_accuracy: 0.9037
Epoch 5/50
Learning rate:  0.001
781/781 [==============================] - 177s 227ms/step - loss: 0.1917 - accuracy: 0.9472 - val_loss: 0.2210 - val_accuracy: 0.9363
Epoch 6/50
Learning rate:  0.001
781/781 [=======================

781/781 [==============================] - 177s 226ms/step - loss: 0.1093 - accuracy: 0.9723 - val_loss: 0.1901 - val_accuracy: 0.9478
Epoch 47/50
Learning rate:  0.001
781/781 [==============================] - 176s 225ms/step - loss: 0.1104 - accuracy: 0.9719 - val_loss: 0.2124 - val_accuracy: 0.9411
Epoch 48/50
Learning rate:  0.001
781/781 [==============================] - 176s 225ms/step - loss: 0.1097 - accuracy: 0.9723 - val_loss: 0.1690 - val_accuracy: 0.9538
Epoch 49/50
Learning rate:  0.001
781/781 [==============================] - 177s 227ms/step - loss: 0.1090 - accuracy: 0.9721 - val_loss: 0.1590 - val_accuracy: 0.9566
Epoch 50/50
Learning rate:  0.001
781/781 [==============================] - 177s 226ms/step - loss: 0.1083 - accuracy: 0.9725 - val_loss: 0.1900 - val_accuracy: 0.9461


In [8]:
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Test loss: 0.18995687561035157
Test accuracy: 0.9460599422454834
